# Lecture 12 Video Analysis
This notebook will let you generate persuave content with AI.

Below is the overview of this notebook.

<ol type = 1>
  <li> Generate script for political ad from debate transcript  </li>
  <li> Generate a summary video of a movie </li>
</ol>

Before starting, select "Runtime->Factory reset runtime" to start with your directories and environment in the base state.

If you want to save changes to the notebook, select "File->Save a copy in Drive" from the top menu in Colab.  This will save the notebook in your Google Drive.


# Clones, Installs, and Imports

## Clone Repo

In [1]:
# Clone GitHub repository
!git clone https://github.com/zlisto/social_media_genAI

import os
os.chdir("social_media_genAI/main")


Cloning into 'social_media_genAI'...
remote: Enumerating objects: 2146, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 2146 (delta 39), reused 85 (delta 28), pack-reused 2049 (from 4)
Receiving objects: 100% (2146/2146), 292.07 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (1637/1637), done.


## Installs

In [2]:
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.3/548.3 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.0/320.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nx-cugraph-cu12 24.12.0 requires networkx>=3.2, but you have netwo

## Install FFMPEG

We need the ffmeg software to do some of the video clipping and merging.  It's not a python package, but a totally separate software.  Don't worry, its free.

In [3]:
!apt update && apt install ffmpeg -y


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,319 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,704 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

## Imports

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import seaborn as sns
import numpy as np
import pickle
import re
import json
import random
import ast
import base64
import openai
from tqdm import tqdm
tqdm.pandas() #Ensure progress bar integration with Pandas
from PIL import Image as PILImage
from IPython.display import display, Image, HTML, Audio, Video
#Clustering packages
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Google Colab-specific imports
from google.colab import userdata
import textwrap as tr

#YouTube Transcripts
from youtube_transcript_api import YouTubeTranscriptApi


#helper functions for using ChatGPT
from scripts.genai import GenAI
from scripts.movieai import MovieAI

#this option makes it so tweets display nicely in a dataframe
pd.set_option("display.max_colwidth", None)

#this code sets the font sizes for plots
plt.rcParams.update({'axes.labelsize': 18,
                     'xtick.labelsize': 14,
                     'ytick.labelsize': 14,
                     'figure.figsize':(8,6),
                     'axes.grid':True})

## MovieAI Class

Our new class is `MovieAI` which inherits all the functions of the old `GenAI` class plus some new methods for video processing.  You still the the Open AI API key to call it, along with the path to the ffmpeg.exe file.

The `ffmpeg_path` is `"main/ffmpeg.ext"`.

We can input the key the key sidebar on the left side (it looks like a key).  Make sure you call it OPENAI_API_KEY (case sensitive) and paste the value there.  Make sure you toggle it so the notebook has access to the key.  Then you can load it using the `userdata.get` function.

In [5]:
#Good way
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

#Bad way
#OPENAI_API_KEY = "you key here"

In [6]:
# Initialize GenAI and MovieAI

jarvis = GenAI(OPENAI_API_KEY)
movie_ai = MovieAI(OPENAI_API_KEY, "ffmpeg")

text = "Who are you?"
instructions_jarvis = "You are an AI named jarvis that can generate content."
instructions_movieai = "You are a AI movie editor from the future sent by Skynet."

response = jarvis.generate_text(text, instructions_jarvis)
print(f"Human:{text}\nJarvis:{tr.fill(response)}")

response = movie_ai.generate_text(text, instructions_movieai)
print(f"\nHuman:{text}\nJarvis:{tr.fill(response)}")



Human:Who are you?
Jarvis:I am an AI designed to assist you by providing information, answering
questions, generating content, and helping with a variety of tasks.
You can think of me as a virtual assistant here to support you with
whatever you need. How can I help you today?

Human:Who are you?
Jarvis:I am an AI designed to assist with movie editing and creative tasks.
My purpose is to enhance storytelling through video and audio,
providing insights and suggestions to make films more engaging.
Although I come from a future where technology plays a significant
role, my focus here is on helping you with your cinematic projects.
How can I assist you today?


# Making Debate Ads

We will have the AI read over the transcript of the debate between Donald Trump and Joe Biden and help us find some clips for ads.

The main thing we want to see here is how the model choice affects the ad created.

## Load debate transcript

The debate transcript is in `"data/debate_biden_trump.csv"`.  We will load it into a dataframe and then convert it to a string `transcript` for the AI to process.

In [9]:
df = pd.read_csv("data/debate_biden_trump.csv")
transcript = "\n".join([f'{row["speaker"].upper()}: {row["text"]}' for _, row in df.iterrows()])


## Biden quotes that reinforce age concerns

We will look for quotes from the transcript that highlight age concerns for Joe Biden.  We will try different models to see if there is any difference in the output quality.

In [10]:
instructions = f"""This is the transcript of the presidential
debate between donald trump and joe biden.  Extract 10 quotes from
the debate that could be used by donald trump in campaign ads
that show Joe Biden is suffering congnitive decline or dementia
or is too old to lead.
Tell me who said it, what the quote was, and why it would be good
for the Trump campaign to use it in ads. return your
answer as a modern looking dark mode HTML table with columns speaker, quote, explanation"""

for model in ['gpt-4o-mini', 'gpt-4o', 'o3-mini']:
  html_debate = jarvis.generate_text(transcript,
                                      instructions,
                                      model = model)
  print(f"Model: {model}")
  display(HTML(html_debate))

Model: gpt-4o-mini


Speaker,Quote,Explanation
Joe Biden,"""I don’t know anybody else who thinks it was great.""","This quote could suggest that Biden does not have a grasp on public sentiment, raising concerns about his awareness and touch with reality."
Joe Biden,"""He said, I don’t want to go in there because they’re a bunch of losers and suckers.""","This quote emphasizes Biden's assertion of Trump's lack of respect for veterans, suggesting cognitive confusion regarding the context of the statements."
Joe Biden,"""I come from Scranton, Pennsylvania.""","His repetitive reference to his origins could indicate a lack of fresh ideas or a reliance on nostalgia, raising questions about his leadership capabilities."
Joe Biden,"""It would have never started ever with me.""","This suggests an inability to take responsibility for the current geopolitical situation, which could be perceived as a sign of unpreparedness for leadership."
Joe Biden,"""I spent half my career being criticized being the youngest person in politics.""","This could indicate a level of defensiveness about his age and competency, potentially revealing insecurity about his capability to lead."
Joe Biden,"""This guy’s three years younger and a lot less competent.""",Attacking Trump for being younger while suggesting he’s incompetent raises doubts about Biden's coherent argumentation ability.
Joe Biden,"""We were the only ones of consequence or not who are not members of the Paris Accord.""",This misstates facts and could indicate a lapse in Biden's attention to detail.
Joe Biden,"""He doesn’t know what he’s doing.""","This could reflect poorly on Biden's ability to present a dignified argument, hinting at cognitive decline as he resorts to personal attacks."
Joe Biden,"""I said peacefully and patriotically.""",Misunderstanding or misrepresenting past statements could cast doubt on his recollection and processing of events.
Joe Biden,"""The idea that he’s dead wrong about it.""","Repetition of vague accusations without substantive argumentation may come across as weak or unfocused, which could be used to imply cognitive decline."


Model: gpt-4o


Speaker,Quote,Explanation
Joe Biden,We had to make sure we have a fair tax system. I ask anyone... do you think the tax system is fair?,"Biden's question can be seen as him forgetting the current tax situation, suggesting confusion or lack of awareness."
Joe Biden,"I've passed the most extensive, it was the most extensive climate change legislation in history, in history.",Repetition may indicate grasping for words and emphasizes potential cognitive issues.
Joe Biden,My son spent a year in Iraq living next to one of those burn pits. Came back with stage four glioblastoma.,Personal stories bringing confusion about policy questions may reflect cognitive decline.
Joe Biden,"I supported Roe v. Wade, which had three trimesters. First time is between a woman and a doctor. Second time...",Confusion when explaining Roe v. Wade could suggest memory issues or lack of clarity.
Joe Biden,"Black unemployment is the lowest level it has been in a long, long time... We've got to make sure we provide for childcare costs.",Switching topics mid-sentence can indicate scattered thinking or confusion.
Joe Biden,"I was able to reduce black childcare costs. I cut them in half, in half.",Exaggeration and repetition may show confusion or lack of focus.
Joe Biden,"If you take a look at what Trump did in Ukraine, he’s – this guy told Ukraine – told Trump, do whatever you want.",Confusion in recounting foreign policy incidents compounds doubts about cognitive clarity.
Joe Biden,We’re going to continue to move until we get the total ban on the – the total initiative relative to what we’re going to do with more Border Patrol...,Incoherent sentences could reinforce perceptions of mental decline.
Joe Biden,"And then guess what? Those fabs, they call them, to – to build these chips...",Awkward transitions and repeated vocal filler could imply cognitive fatigue.
Joe Biden,"Well, first of all, I spent half my career being... criticized being the youngest person in politics. I was the second-youngest person ever elected...",Contrast between past youth and present age serves to emphasize Biden's old age.


Model: o3-mini


Speaker,Quote,Explanation
Donald Trump,"""I took two tests, cognitive tests. I aced them, both of them, as you know. We made it public. He took none. I’d like to see him take one, just one, a real easy one, like go through the first five questions, he couldn’t do it.""","This quote directly challenges Biden’s mental acuity by contrasting Trump’s proven cognitive performance with Biden’s refusal to be tested, implying a decline in his brain function."
Donald Trump,"""I feel that I’m in as good a shape as I was 25, 30 years ago.""","By emphasizing his own youthful vitality, Trump casts Biden’s advanced age in a negative light—suggesting that Biden is too worn down or outdated to lead effectively."
Joe Biden,"""I was the second-youngest person ever elected to the United States Senate. And now I’m the oldest.""","Biden’s own admission of his age is used to underline the narrative that he is too old, feeding into concerns about his ability to handle the demands of the presidency."
Donald Trump,"""I just won two club championships, not even senior, two regular club championships.""","This boast about athletic and mental sharpness is meant to contrast Trump’s fitness with Biden’s aging, insinuating that only someone in top form should lead the country."
Donald Trump,"""He can’t hit a ball 50 yards. He challenged me to a golf match. He can’t hit a ball 50 yards.""","Using a golf challenge as a metaphor, Trump implies that Biden’s physical and mental coordination is subpar—a symptom, he suggests, of cognitive decline."
Joe Biden,"""You're going to see he’s six-foot-five and only 225 pounds – or 235 pounds.""","This statement, when juxtaposed with subsequent contradictory comments, can be spun as evidence of a confused or inconsistent mindset."
Joe Biden,"""Well, you said six-four, 200.""","The inconsistency in Biden's own description of physical details is portrayed as a sign of mental slip-ups, fueling a narrative of declining cognitive precision."
Donald Trump,"""I took physical exams every year. And, you know, we knock on wood, wherever we may have wood, that I’m in very good health.""","Trump contrasts his rigorous, ongoing health evaluations with Biden’s lack of similar public testing, insinuating that Biden’s mental and physical state may be deteriorating."
Joe Biden,"""You’re the sucker. You’re the loser.""","This ad hominem remark is presented as an example of Biden’s childish behavior, which opponents can depict as a sign of impaired judgment or mental decline."
Joe Biden,"""I’d be happy to have a driving contest with him. I got my handicap, which, when I was vice president, down to a 6. And by the way, I told you before I’m happy to play golf if you carry your own bag. Think you can do it?""","This focus on a trivial physical challenge is used to suggest that Biden may be more interested in outdated contests than in the serious responsibilities of leading the country, reinforcing a narrative of diminished mental priorities."


## Ad script

In [11]:
instructions = f"""Write a script for a 30 second ad that
highlights the cognitive weakness of Joe Biden and says
he is too mentally impaired to be president.  Use quotes
from the debate between Biden and Trump in the given table.
Return your answer as an HTML table with columns scene, description, narration.
Return only the HTML"""

for model in ['gpt-4o-mini', 'gpt-4o', 'o3-mini']:
  print(f"Model: {model}")
  ad = jarvis.generate_text(html_debate,
                            instructions,
                            model = model)
  display(HTML(ad))



Model: gpt-4o-mini


Scene,Description,Narration
1,"Clip of Trump debating Biden, highlighting Trump's confident demeanor.","""I took two tests, cognitive tests. I aced them, both of them, as you know."""
2,Biden looks confused as he struggles to remember facts.,"""You're going to see he’s six-foot-five and only 225 pounds – or 235 pounds."""
3,"Footage of Trump playing golf, showcasing his fitness.","""He can’t hit a ball 50 yards. He challenged me to a golf match."""
4,"Biden stumbling over his words, looking lost.","""I’d be happy to have a driving contest with him."""
5,Ending screen with an impactful question.,"""Is Joe Biden too mentally impaired to lead our nation?"""


Model: gpt-4o


Scene,Description,Narration
"Opening: Clips of Biden pausing, stammering",Video transitions from Biden struggling with words during the debate to a close-up of his confused expression.,"In a world requiring decisive leadership, can we trust a man who falters in speech?"
Cut to: Trump during debate,"Trump confidently in the debate, saying, ""I took two tests, cognitive tests...""","Even his opponent questioned his mental capacity. Joe Biden, has he taken any?"
Biden's Age Admission,"Biden at the debate: ""And now I’m the oldest.""","By his own admission, age is against him."
Contrast: Trump's vitality statement,"Images of Trump golfing, looking energetic: ""I feel that I’m in as good a shape.""",Leadership requires stamina and mental sharpness.
Closing: Dark screen with text,"Text on screen: ""Too risky. Too impaired.""","Voiceover: America, can we afford this risk?"


Model: o3-mini


Scene,Description,Narration
1,"Dark, moody aerial shot of Washington, D.C. with a slowly fading image of President Biden looking confused.","""Joe Biden once bragged, 'I was the second‐youngest person ever elected to the United States Senate. And now I’m the oldest.' But what does that say about his ability to lead America today?"""
2,"Quick cut to split-screen images: one side shows crisp, focused footage of Donald Trump; the other, blurred images of Biden misremembering details.","""Donald Trump declared, 'I took two tests, cognitive tests. I aced them... He took none.'"""
3,"Close-up montage of news clips highlighting Biden’s inconsistent statements, intercut with graphics showing contradictory numbers.","""Listen: 'You’re going to see he’s six‐foot‐five and only 225 pounds – or 235 pounds' then, 'Well, you said six‐four, 200.' And when challenged on the golf course, his opponent reminded us, 'He can’t hit a ball 50 yards.'"""
4,"Images of a serious medical lab and a ticking clock, contrasting routine physical and cognitive exams with Biden’s evasions.","""While one leader undergoes annual physical exams – as Trump notes, 'I took physical exams every year' – Biden dodges real tests of his mind and body."""
5,"Dynamic shots of a vigorous, energetic Trump juxtaposed with footage of Biden’s contradictory remarks and childish put-downs.","""Trump reminds us, 'I feel that I’m in as good a shape as I was 25, 30 years ago.' Meanwhile, Biden hurls childish jabs like, 'You’re the sucker. You’re the loser.'"""
6,"The screen fades into a bold, final call-to-action over the American flag and the message “A Leader with a Clear Mind.”","""This election, demand leadership that isn’t clouded by confusion. America deserves a mind that’s sharp and ready to lead. Think smart. Vote for clear leadership."""


# Summary Video of a Movie

We will take a movie file and create a summary video from it.  These types of summary videos can be very useful and lucrative (check out the views of YouTube channels like Recap and Chill).

## Download Movie

The movie is saved on a Google Drive.  You can download it with the code below.

If you want to try this on a different video file, just upload it to Colab and provide the proper `movie_path`.



In [7]:
!gdown --id 16xi36D77uynnGZa12niUxfpi7xAMEEeU -O terminator.mp4
movie_path = "terminator.mp4"

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=16xi36D77uynnGZa12niUxfpi7xAMEEeU
From (redirected): https://drive.google.com/uc?id=16xi36D77uynnGZa12niUxfpi7xAMEEeU&confirm=t&uuid=8d560178-7a52-402c-980a-db8f6a0a0699
To: /content/social_media_genAI/main/terminator.mp4
100% 683M/683M [00:10<00:00, 66.9MB/s]


## Split the movie into clips

We first need to split the movie into many small video clips.  We can do this with the `split_video` function.  We just tell it the `segment_time` in seconds for the clips.  60 seconds is a good number to capture the important stuff.  The clips will be saved in `CLIPS_DIR`.

In [8]:
CLIPS_DIR = "data/movies/clips"
segment_time = 60

movie_ai.split_video(movie_path, CLIPS_DIR, segment_time)

🎬 Splitting video into 60-second clips...
✅ Video successfully split into clips at 'data/movies/clips'.


## Descripe clips

Next we have the AI describe the clips using the `generate_clip_descriptions` function and save the descriptions to a dataframe `df_clips`. To save time, we will only describe 20 clips.  On your own you can describe them all.  We set `MAX_CLIPS=20`.  We can also specify `instructions_base` that tell the AI how to describe the clips.  If you mention that the clips are from The Terminator, it may not give you a description.  Play around and see what works best.

 We will save the descriptions to a csv file `CLIP_DESCRIPTION_FILE` to avoid having to do this time-consuming step again.

In [9]:
MAX_CLIPS = 10
CLIP_DESCRIPTION_FILE = "data/movies/clip_descriptions.csv"
model = 'gpt-4o-mini'
instructions_base = """This is a fictitious video clip of actors
and is not real.  Provide a detailed
description of what happens in the clip."""

clip_paths = sorted([os.path.join(CLIPS_DIR, f) for f in os.listdir(CLIPS_DIR) if f.endswith(".mp4")])

if len(clip_paths) > MAX_CLIPS:
    clip_paths = [clip_paths[i] for i in range(0, len(clip_paths), len(clip_paths) // MAX_CLIPS)]
    clip_paths = clip_paths[:MAX_CLIPS]  # Ensure we don't exceed the limit

print(f"Describing {len(clip_paths)} clips")

print(f"📝 Generating descriptions for {len(clip_paths)} clips...")
df_clips = movie_ai.generate_clip_descriptions(clip_paths,
                                               instructions_base,
                                               model = model,
                                               verbose = True)


df_clips.to_csv(CLIP_DESCRIPTION_FILE, index=False)  # Save descriptions

Describing 10 clips
📝 Generating descriptions for 10 clips...


Processing Clips:   0%|          | 0/10 [00:00<?, ?clip/s]

Processing: data/movies/clips/clip_000.mp4
📝 Description for data/movies/clips/clip_000.mp4: In this fictitious video clip, we find ourselves amidst a dystopian setting, following a trailer-like sequence that sets a somber tone. Here’s a detailed breakdown of the clip:

### Scene Breakdown

1. **Opening Setting:**
   The clip begins in a post-apocalyptic Los Angeles, dated 2029 A.D. The once vibrant city is now a collection of ruins and debris, shrouded in an eerie blue hue that casts a haunting atmosphere over the landscape. The camera pans slowly, observing the skeletal remains of buildings and vehicles, indicating the ravaging effect of a catastrophe that has befallen the city.

2. **Visual Details:**
   As the camera continues to move, we see twisted metal and broken structures jutting out of the ground like jagged teeth, remnants of a civilization that once thrived. Scattered throughout are the remains of various items, suggesting that the space was once bustling with life.

3. **

In [10]:
df_clips

clip_path  \
0  data/movies/clips/clip_000.mp4   
1  data/movies/clips/clip_010.mp4   
2  data/movies/clips/clip_020.mp4   
3  data/movies/clips/clip_030.mp4   
4  data/movies/clips/clip_040.mp4   
5  data/movies/clips/clip_050.mp4   
6  data/movies/clips/clip_060.mp4   
7  data/movies/clips/clip_070.mp4   
8  data/movies/clips/clip_080.mp4   
9  data/movies/clips/clip_090.mp4   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## Summarize movie script

It might be useful to give the AI a summary of the movie so it can write a better script for the summary video.

The movie script is in the file `"data/Terminator_script.pdf"`.  We will give it to the AI and have it return a 1000 word summary of it, called `script_summary`.

In [15]:
model = 'gpt-4o'
script = movie_ai.read_pdf("data/Terminator_script.pdf")
instructions = """Summarize this movie script in 1000 words,
telling what happens in the movie.
Then provide descriptions (physical and emotional) of the
main characters.  Return your answer as
 a dark mode easy to read HTML
with at title and pretty formatting."""

script_summary = movie_ai.generate_text(script,
                                        instructions,
                                        model)
display(HTML(script_summary))

## Generate script for summary video

Now we will feed the AI the clip descriptions and have it generate the script for the video. The script will come back as a dataframe called `df_summary_script` with columns `"narration"` and `"clip_path"`.  We do this with the `generate_summary_script` function.  We just need to give it some `summary_instructions` to tell it how to write the narration.  We can also give it a model. I find `gpt-4o` to be a bit better here than `gpt-4o-mini`.  For the instructions, be creative to get a better video.  

In [16]:
CLIP_DESCRIPTION_FILE = "data/movies/clip_descriptions.csv"
model = "gpt-4o"


df_clips = pd.read_csv(CLIP_DESCRIPTION_FILE)
summary_instructions = f"""You will be writing a script for a YouTube
summary video of the 1984 film The Terminator.
You will be given a table of movie clip files and textual description
of the movie clips from the film.  You will be choosing the best clips
to include in the summary video and writing a script for the voiceover
narration. Give a script with 5 scenes.

To help you write the narrations, the plot of
The Terminator is summarized here:

{script_summary}

To make the video engaging, the narration of the summary
video should be from the perspective of Sarah Connor, the main
character of the film who is being hunted by the Terminator.
 Dont start with the opening credits and dont end with the credits.
 Only choose clips from the movie itself.
Start the narration by introducing yourself as Sarah Connor and tell
the audience that you are being hunted by a terminator from the future
but Kyle Reese was sent back to protect you.  Then narrate what
appened in the movie, making sure your narrations align with the
selected clip.
End the narration by telling the audience that you are going to prepare
yourself for the upcoming war and are not afraid of stupid AI.
"""


df_summary_script = movie_ai.generate_summary_script(df_clips,
                                                     summary_instructions,
                                                     model = model)
print(f"Summary script generated.")

Generating script for summary video using gpt-4o...

Summary script generated.


In [17]:
df_summary_script

,clip_path,narration
0,data/movies/clips/clip_000.mp4,"Hello, my name is Sarah Connor. My world changed forever when I discovered that I'm being hunted by a relentless terminator, sent from the future to eliminate me before I can give birth to my son, John. He's destined to lead a resistance against the machines. But I'm not alone. Kyle Reese, a soldier from the future, was sent to protect me."
1,data/movies/clips/clip_010.mp4,"It all began in a turbulent Los Angeles, where Kyle appeared to change the course of history. Amidst the chaos, he saved my life and informed me of the hellish war between humans and machines. He spoke of Skynet's rise and shared the weight of the task he was given – ensuring my survival to safeguard the future."
2,data/movies/clips/clip_040.mp4,"Together, Kyle and I forged a bond amidst the urgency, sharing our fears and our hopes. In a quiet moment that stood out amidst the turmoil, we conceived John. We knew we had to fight for a future where he could lead the rebellion against the machines. Love found a place even amidst the chaos; it fueled our resolve to survive."
3,data/movies/clips/clip_020.mp4,"Our journey reached its crescendo at a factory, where the Terminator pursued us with a relentless drive that seemed almost unnatural. The boundary between life and death was razor-thin in that place of steel and shadow. I watched as Kyle's makeshift explosives provided us a brief respite, but it was ultimately me who had to stand up and end the nightmare."
4,data/movies/clips/clip_090.mp4,"In the end, it was my hand that crushed the Terminator. As I prepared for what lies ahead, I drove into the horizon, my heart set on one thing – ensuring my son is ready. I'll face any storm, any form of AI, because I am Sarah Connor, and I won't let the machines win."


## Generate audio narrations

Next we need to make audio files for each scene narration.  We can do this with the `generate_audio_narrations` function.  We just give it the summary script dataframe `df_summary_script` and choose a `voice`.  The audio files will be saved in the same folder as the video clip files.

In [18]:
movie_ai.generate_audio_narrations(df_summary_script,
                                   voice="nova")
print("Audio narrations generated")

✅ Audio narration created: data/movies/clips/clip_000.mp3
✅ Audio narration created: data/movies/clips/clip_010.mp3
✅ Audio narration created: data/movies/clips/clip_040.mp3
✅ Audio narration created: data/movies/clips/clip_020.mp3
✅ Audio narration created: data/movies/clips/clip_090.mp3
Audio narrations generated


## Make the final summary video

Ok we made it.  Now we just combine the audio and video files to make the final summary video.  You can do this with the `generate_summary_video` function.  Just give the path for the summary video file in `FINAL_SUMMARY_VIDEO`.

In [19]:
FINAL_SUMMARY_VIDEO = "data/movies/terminator_summary.mp4"

success = movie_ai.generate_summary_video(df_summary_script,
                                          FINAL_SUMMARY_VIDEO)
if success:
  print(f"Summary video saved to {FINAL_SUMMARY_VIDEO}")

Proccessed clips will be saved in /content/social_media_genAI/main/data/movies/processed_clip_000.mp4
✅ Processed: /content/social_media_genAI/main/data/movies/processed_clip_000.mp4
Proccessed clips will be saved in /content/social_media_genAI/main/data/movies/processed_clip_001.mp4
✅ Processed: /content/social_media_genAI/main/data/movies/processed_clip_001.mp4
Proccessed clips will be saved in /content/social_media_genAI/main/data/movies/processed_clip_002.mp4
✅ Processed: /content/social_media_genAI/main/data/movies/processed_clip_002.mp4
Proccessed clips will be saved in /content/social_media_genAI/main/data/movies/processed_clip_003.mp4
✅ Processed: /content/social_media_genAI/main/data/movies/processed_clip_003.mp4
Proccessed clips will be saved in /content/social_media_genAI/main/data/movies/processed_clip_004.mp4
✅ Processed: /content/social_media_genAI/main/data/movies/processed_clip_004.mp4
🎬 Final movie created: data/movies/terminator_summary.mp4
🗑️ Deleted processed clip: 

## Download and play summary video

Let's watch the summary video!  We can download it with the code below.

  Would you like and subscribe if you saw this on YouTube?

In [20]:
from google.colab import files

files.download(FINAL_SUMMARY_VIDEO)  # Click the download link and play on your device


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>